# ABACO Financial Intelligence Platform - Comprehensive Analytics

## Enterprise-Grade Financial Analysis System with Multi-Agent Orchestration

This notebook implements a complete financial intelligence platform with:
- Structured JSON logging with automatic context tracking
- Robust file ingestion with column normalization
- **Multi-Agent System**: Supervisor orchestrates Data Extraction and Quantitative Analysis agents
- Advanced KPI calculation with deeper segmentation and anomaly detection
- **Hybrid Alert System**: Rule-based detection + AI-powered severity classification
- **Advanced Growth Models**: ARIMA forecasting, Monte Carlo simulations, scenario analysis
- Roll-rate and delinquency cascade analysis
- Marketing and sales analytics with Treemap visualization
- AI-powered insights generation
- Production-ready exports for Figma integration

---
**Version:** 3.1.0 (Multi-Agent Enhanced)  
**Platform:** ABACO Financial Intelligence  
**Data Framework:** Enterprise-grade with agent orchestration

## 1. Environment Setup & Structured Logging

Initialize production environment with JSON-formatted logging, session state management, and file upload capabilities. This cell ensures idempotent operations and prevents data duplication on re-runs.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import json
import logging
import time
import re
import io
import uuid
from datetime import datetime, timezone, timedelta
from pathlib import Path
from typing import Dict, List, Any, Optional, Tuple, Union
from functools import lru_cache
import warnings
warnings.filterwarnings('ignore')

try:
    import plotly.graph_objects as go
    import plotly.express as px
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False

try:
    import vertexai
    from vertexai.preview.generative_models import GenerativeModel
    AI_AVAILABLE = True
except ImportError:
    AI_AVAILABLE = False

class StructuredLogger:
    def __init__(self, session_id: str):
        self.session_id = session_id
        self.logs = []
        self.handlers = []

    def log(self, level: str, operation: str, status: str, message: str = "", 
            metadata: Optional[Dict] = None, error: Optional[str] = None):
        log_entry = {
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "session_id": self.session_id,
            "level": level,
            "operation": operation,
            "status": status,
            "message": message,
            "metadata": metadata or {},
            "error": error
        }
        self.logs.append(log_entry)
        print(json.dumps(log_entry))

    def get_summary(self) -> Dict[str, Any]:
        return {
            "session_id": self.session_id,
            "total_logs": len(self.logs),
            "by_level": self._count_by_field("level"),
            "by_operation": self._count_by_field("operation"),
            "errors": [l for l in self.logs if l.get("error")]
        }

    def _count_by_field(self, field: str) -> Dict[str, int]:
        counts = {}
        for log in self.logs:
            key = log.get(field)
            counts[key] = counts.get(key, 0) + 1
        return counts

session_id = str(uuid.uuid4())[:8]
logger = StructuredLogger(session_id)

logger.log("INFO", "environment_setup", "start", 
          message="Initializing ABACO Financial Intelligence Platform",
          metadata={"session_id": session_id})

## 2. Robust File Ingestion & Column Normalization

Handle file uploads with support for CSV, XLSX, and PDF formats. Normalize column names to lowercase with underscore-separated spaces and special characters. Convert numeric values tolerant of currency symbols (₡, $, €), commas, and percentage signs. Maintain ingestion state to prevent duplication on re-runs.

In [ ]:
class RobustDataIngestion:
    def __init__(self, logger: StructuredLogger):
        self.logger = logger
        self.ingestion_state = {"shapes": {}, "loaded_flags": {}, "data": {}}
        self.currency_symbols = r"[\₡$€,%\s]"

    def normalize_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        df.columns = (
            df.columns.str.lower()
            .str.strip()
            .str.replace(r"\s+", "_", regex=True)
            .str.replace(r"[^a-z0-9_]", "", regex=True)
        )
        return df

    def convert_numeric_tolerant(self, series: pd.Series) -> pd.Series:
        if series.dtype in ["int64", "float64"]:
            return series
        
        series_str = series.astype(str)
        series_cleaned = (
            series_str.str.replace(self.currency_symbols, "", regex=True)
            .str.replace(",", "", regex=True)
            .str.strip()
        )
        return pd.to_numeric(series_cleaned, errors="coerce")

    def parse_document(self, filepath: str) -> Union[pd.DataFrame, Dict[str, pd.DataFrame], None]:
        try:
            if filepath.endswith(".csv"):
                return pd.read_csv(filepath)
            elif filepath.endswith(".xlsx"):
                excel_file = pd.ExcelFile(filepath)
                return {sheet: pd.read_excel(filepath, sheet_name=sheet) 
                       for sheet in excel_file.sheet_names}
            elif filepath.endswith(".pdf"):
                try:
                    import PyPDF2
                    with open(filepath, "rb") as f:
                        reader = PyPDF2.PdfReader(f)
                        return {"text": "".join(page.extract_text() for page in reader.pages)}
                except ImportError:
                    self.logger.log("WARN", "parse_document", "skip", 
                                   message="PyPDF2 not available for PDF parsing")
                    return None
        except Exception as e:
            self.logger.log("ERROR", "parse_document", "failed", 
                           message=f"Error parsing {filepath}", error=str(e))
        return None

    def ingest_file(self, filepath: str) -> bool:
        filename = Path(filepath).name
        
        if filename in self.ingestion_state["loaded_flags"]:
            self.logger.log("INFO", "ingest_file", "skip", 
                           message=f"File {filename} already loaded")
            return True

        data = self.parse_document(filepath)
        if data is None:
            return False

        if isinstance(data, dict):
            for sheet, df in data.items():
                if isinstance(df, pd.DataFrame):
                    df = self.normalize_columns(df)
                    for col in df.columns:
                        df[col] = self.convert_numeric_tolerant(df[col])
                    key = f"{filename}_{sheet}"
                    self.ingestion_state["data"][key] = df
                    self.ingestion_state["shapes"][key] = df.shape
        elif isinstance(data, pd.DataFrame):
            data = self.normalize_columns(data)
            for col in data.columns:
                data[col] = self.convert_numeric_tolerant(data[col])
            self.ingestion_state["data"][filename] = data
            self.ingestion_state["shapes"][filename] = data.shape

        self.ingestion_state["loaded_flags"][filename] = True
        self.logger.log("INFO", "ingest_file", "success", 
                       message=f"Ingested {filename}",
                       metadata=self.ingestion_state["shapes"])
        return True

ingestion = RobustDataIngestion(logger)

data_directory = Path("/workspaces/nextjs-with-supabase/data")
if data_directory.exists():
    for filepath in data_directory.glob("**/*.[!.]*"):
        if filepath.suffix.lower() in [".csv", ".xlsx", ".pdf"]:
            ingestion.ingest_file(str(filepath))

logger.log("INFO", "data_ingestion", "complete",
         message="Data ingestion phase complete",
         metadata={"files_loaded": len(ingestion.ingestion_state["loaded_flags"])})

data_sources = ingestion.ingestion_state["data"]
print(f"\nLoaded data sources: {list(data_sources.keys())}")

## 3. Comprehensive KPI Calculation Engine

Calculate all financial metrics and KPIs across all customer segments, regions, industries, and product lines. Generate comprehensive business views including:
- Portfolio overview metrics
- Risk metrics and default rates
- Segment-level analytics
- Delinquency distribution
- Regional and industry breakdowns
- Alerts with probability scoring

In [ ]:
class ComprehensiveKPIEngine:
    def __init__(self, data: pd.DataFrame, logger: StructuredLogger):
        self.data = data.copy()
        self.logger = logger
        self.trace_id = str(uuid.uuid4())[:8]
        self.operation_start = datetime.now(timezone.utc)

    def validate_required_columns(self) -> bool:
        required = {"balance", "creditlimit", "dayspastdue", "customersegment"}
        available = set(self.data.columns)
        missing = required - available
        
        if missing:
            self.logger.log("ERROR", "kpi_validation", "failed",
                           message=f"Missing required columns: {missing}")
            return False
        return True

    def calculate_all_kpis(self) -> Dict[str, Any]:
        if self.data.empty:
            self.logger.log("WARN", "kpi_calculation", "empty_data")
            return {"status": "empty_data", "kpis": {}}

        if not self.validate_required_columns():
            return {"status": "invalid_columns", "kpis": {}}

        self.logger.log("INFO", "kpi_calculation", "start",
                       message=f"Calculating KPIs for {len(self.data)} records",
                       metadata={"trace_id": self.trace_id})

        self._calculate_derived_metrics()
        
        kpis = {
            "portfolio_overview": self._portfolio_overview(),
            "risk_metrics": self._risk_metrics(),
            "segment_analysis": self._segment_analysis(),
            "delinquency_buckets": self._delinquency_buckets(),
            "dimensional_analysis": self._dimensional_analysis(),
            "alerts": self._generate_alerts()
        }

        duration = (datetime.now(timezone.utc) - self.operation_start).total_seconds()
        self.logger.log("INFO", "kpi_calculation", "complete",
                       message="KPI calculation completed",
                       metadata={"trace_id": self.trace_id, "duration_seconds": duration})

        return {"status": "success", "kpis": kpis}

    def _calculate_derived_metrics(self):
        self.data["utilization_ratio"] = (
            self.data["balance"] / self.data["creditlimit"]
        ).fillna(0).clip(0, 1)

        self.data["dpd_bucket"] = pd.cut(
            self.data["dayspastdue"],
            bins=[-np.inf, 0, 30, 60, 90, np.inf],
            labels=["Current", "1-30 DPD", "31-60 DPD", "61-90 DPD", "90+ DPD"]
        )

    def _portfolio_overview(self) -> Dict[str, float]:
        return {
            "total_aum": float(self.data["balance"].sum()),
            "active_customers": int(len(self.data)),
            "total_credit_lines": float(self.data["creditlimit"].sum()),
            "avg_balance": float(self.data["balance"].mean()),
            "avg_credit_limit": float(self.data["creditlimit"].mean()),
            "portfolio_utilization": float(self.data["utilization_ratio"].mean())
        }

    def _risk_metrics(self) -> Dict[str, float]:
        return {
            "default_rate_30plus": float((self.data["dayspastdue"] >= 30).sum() / len(self.data)),
            "default_rate_90plus": float((self.data["dayspastdue"] >= 90).sum() / len(self.data)),
            "avg_days_past_due": float(self.data["dayspastdue"].mean()),
            "high_utilization_customers": int((self.data["utilization_ratio"] > 0.8).sum()),
            "delinquent_aum": float(self.data[self.data["dayspastdue"] > 0]["balance"].sum())
        }

    def _segment_analysis(self) -> Dict[str, Dict[str, float]]:
        segment_col = next((c for c in self.data.columns if "segment" in c.lower()), None)
        if not segment_col:
            return {}
        
        segments = {}
        for segment in self.data[segment_col].unique():
            segment_data = self.data[self.data[segment_col] == segment]
            segments[str(segment)] = {
                "count": int(len(segment_data)),
                "aum": float(segment_data["balance"].sum()),
                "avg_balance": float(segment_data["balance"].mean()),
                "default_rate_90plus": float((segment_data["dayspastdue"] >= 90).sum() / len(segment_data))
            }
        return segments

    def _delinquency_buckets(self) -> Dict[str, int]:
        return self.data["dpd_bucket"].value_counts().to_dict()

    def _dimensional_analysis(self) -> Dict[str, Dict[str, int]]:
        analysis = {}
        for col in self.data.columns:
            if self.data[col].nunique() < 50 and self.data[col].dtype == "object":
                analysis[col] = self.data[col].value_counts().head(10).to_dict()
        return analysis

    def _generate_alerts(self) -> pd.DataFrame:
        alerts = []

        default_rate_90 = (self.data["dayspastdue"] >= 90).sum() / len(self.data)
        alerts.append({
            "variable": "default_rate_90plus",
            "value": default_rate_90,
            "threshold": 0.05,
            "severity": "high" if default_rate_90 > 0.05 else "low",
            "probability": min(default_rate_90 / 0.05, 1.0)
        })

        high_util = (self.data["utilization_ratio"] > 0.8).sum() / len(self.data)
        alerts.append({
            "variable": "high_utilization_ratio",
            "value": high_util,
            "threshold": 0.2,
            "severity": "medium" if high_util > 0.2 else "low",
            "probability": min(high_util / 0.2, 1.0)
        })

        return pd.DataFrame(alerts)

if data_sources:
    for source_name, df in data_sources.items():
        if isinstance(df, pd.DataFrame) and len(df) > 0:
            kpi_engine = ComprehensiveKPIEngine(df, logger)
            result = kpi_engine.calculate_all_kpis()
            
            if result["status"] == "success":
                kpis = result["kpis"]
                print(f"\n{source_name} - Portfolio Overview:")
                for key, value in kpis["portfolio_overview"].items():
                    print(f"  {key}: {value}")
                break
else:
    logger.log("WARN", "kpi_calculation", "no_data",
              message="No valid data sources available for KPI calculation")

## 4. Data Quality Audit & Validation

Comprehensive data quality assessment with penalty scoring for missing values, zero-row tables, and critical column absence. Identify data anomalies and provide actionable recommendations for data improvement.

In [ ]:
class DataQualityAudit:
    def __init__(self, data_sources: Dict[str, pd.DataFrame], logger: StructuredLogger):
        self.data_sources = data_sources
        self.logger = logger
        self.audit_results = {}

    def run_audit(self) -> Dict[str, Any]:
        self.logger.log("INFO", "quality_audit", "start",
                       message=f"Starting audit of {len(self.data_sources)} sources")

        for source_name, df in self.data_sources.items():
            if not isinstance(df, pd.DataFrame):
                self.audit_results[source_name] = self._audit_non_dataframe(source_name, df)
            else:
                self.audit_results[source_name] = self._audit_dataframe(source_name, df)

        overall_score = self._calculate_overall_score()
        
        self.logger.log("INFO", "quality_audit", "complete",
                       message="Data quality audit complete",
                       metadata={"overall_score": overall_score})

        return {
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "overall_score": overall_score,
            "by_source": self.audit_results
        }

    def _audit_dataframe(self, source_name: str, df: pd.DataFrame) -> Dict[str, Any]:
        score = 100
        issues = []

        if len(df) == 0:
            score -= 50
            issues.append("Empty table")

        missing_pct = (df.isnull().sum() / len(df) * 100).to_dict()
        for col, pct in missing_pct.items():
            if pct > 50:
                score -= 10
                issues.append(f"Column {col} has {pct:.1f}% missing values")
            elif pct > 20:
                score -= 5

        duplicate_pct = (len(df) - len(df.drop_duplicates())) / len(df) * 100 if len(df) > 0 else 0
        if duplicate_pct > 10:
            score -= 15
            issues.append(f"{duplicate_pct:.1f}% duplicate rows")

        return {
            "shape": df.shape,
            "quality_score": max(0, score),
            "issues": issues,
            "column_count": len(df.columns),
            "row_count": len(df)
        }

    def _audit_non_dataframe(self, source_name: str, data: Any) -> Dict[str, Any]:
        return {
            "type": type(data).__name__,
            "quality_score": 50,
            "issues": ["Data is not a DataFrame"]
        }

    def _calculate_overall_score(self) -> float:
        scores = [r.get("quality_score", 0) for r in self.audit_results.values()]
        return np.mean(scores) if scores else 0

audit = DataQualityAudit(data_sources, logger)
audit_report = audit.run_audit()

print(f"\nData Quality Report:")
print(f"Overall Quality Score: {audit_report['overall_score']:.1f}/100")
for source_name, audit_data in audit_report['by_source'].items():
    print(f"\n{source_name}:")
    print(f"  Score: {audit_data.get('quality_score', 'N/A')}")
    if audit_data.get('issues'):
        for issue in audit_data['issues']:
            print(f"  - {issue}")

## 5. Growth Analysis & Projections

Calculate current portfolio metrics and project future growth based on user-defined targets. Perform monthly interpolation and generate gap analysis to identify growth opportunities.

In [ ]:
class GrowthAnalysisEngine:
    def __init__(self, df: pd.DataFrame, logger: StructuredLogger):
        self.df = df
        self.logger = logger

    def calculate_growth_projection(self, target_growth_rate: float, 
                                   projection_months: int = 24) -> Dict[str, Any]:
        self.logger.log("INFO", "growth_analysis", "start",
                       message=f"Starting growth projection with {target_growth_rate}% target",
                       metadata={"projection_months": projection_months})

        current_aum = float(self.df["balance"].sum())
        current_customers = len(self.df)

        target_aum = current_aum * (1 + target_growth_rate / 100)
        target_customers = current_customers * (1 + target_growth_rate / 100)

        months = np.arange(0, projection_months + 1)
        monthly_aum = np.linspace(current_aum, target_aum, len(months))
        monthly_customers = np.linspace(current_customers, target_customers, len(months))

        projection_df = pd.DataFrame({
            "month": months,
            "projected_aum": monthly_aum,
            "projected_customers": monthly_customers.astype(int),
            "gap_aum": target_aum - monthly_aum,
            "gap_customers": target_customers - monthly_customers
        })

        self.logger.log("INFO", "growth_analysis", "complete",
                       message="Growth projection complete",
                       metadata={
                           "current_aum": current_aum,
                           "target_aum": target_aum,
                           "growth_gap": target_aum - current_aum
                       })

        return {
            "current_state": {
                "aum": current_aum,
                "customers": current_customers
            },
            "target_state": {
                "aum": target_aum,
                "customers": target_customers
            },
            "projection": projection_df.to_dict(orient="records")
        }

target_growth_rate = 15.0

if data_sources:
    for source_name, df in data_sources.items():
        if isinstance(df, pd.DataFrame) and len(df) > 0:
            if "balance" in df.columns:
                growth_engine = GrowthAnalysisEngine(df, logger)
                growth_result = growth_engine.calculate_growth_projection(target_growth_rate)
                
                print(f"\nGrowth Analysis for {source_name}:")
                print(f"Current AUM: ${growth_result['current_state']['aum']:,.0f}")
                print(f"Target AUM (with {target_growth_rate}% growth): ${growth_result['target_state']['aum']:,.0f}")
                print(f"Growth Gap: ${growth_result['target_state']['aum'] - growth_result['current_state']['aum']:,.0f}")
                break

## 6. Roll-Rate & Delinquency Cascade Analysis

Analyze historical delinquency transitions and cascade patterns. Track customer movement through delinquency buckets to identify risk trends and recovery patterns.

In [ ]:
class RollRateAnalysis:
    def __init__(self, df: pd.DataFrame, logger: StructuredLogger):
        self.df = df
        self.logger = logger

    def calculate_roll_rates(self) -> Dict[str, Any]:
        self.logger.log("INFO", "rollrate_analysis", "start")

        if "dayspastdue" not in self.df.columns:
            self.logger.log("WARN", "rollrate_analysis", "skip",
                           message="DPD column not found")
            return {}

        buckets = pd.cut(
            self.df["dayspastdue"],
            bins=[-np.inf, 0, 30, 60, 90, np.inf],
            labels=["Current", "1-30", "31-60", "61-90", "90+"]
        )

        bucket_dist = buckets.value_counts(normalize=True).sort_index()
        
        roll_rates = {}
        for bucket_name in ["Current", "1-30", "31-60", "61-90", "90+"]:
            bucket_data = self.df[buckets == bucket_name]
            if len(bucket_data) > 0:
                pct_in_bucket = len(bucket_data) / len(self.df)
                roll_rates[bucket_name] = {
                    "count": int(len(bucket_data)),
                    "percentage": float(pct_in_bucket),
                    "aum": float(bucket_data["balance"].sum())
                }

        self.logger.log("INFO", "rollrate_analysis", "complete")
        
        return roll_rates

if data_sources:
    for source_name, df in data_sources.items():
        if isinstance(df, pd.DataFrame) and len(df) > 0:
            if "dayspastdue" in df.columns:
                rollrate = RollRateAnalysis(df, logger)
                roll_result = rollrate.calculate_roll_rates()
                
                print(f"\nRoll-Rate Analysis for {source_name}:")
                for bucket, metrics in roll_result.items():
                    print(f"  {bucket}: {metrics['count']} customers ({metrics['percentage']*100:.1f}%)")
                break

## 7. Marketing & Sales Analysis with Treemap Visualization

Analyze portfolio distribution across industries, regions, and products. Generate hierarchical visualizations and aggregate performance metrics for strategic planning.

In [ ]:
class MarketingAnalyticsEngine:
    def __init__(self, df: pd.DataFrame, logger: StructuredLogger):
        self.df = df
        self.logger = logger

    def analyze_distribution(self, dimensions: List[str]) -> Dict[str, Any]:
        self.logger.log("INFO", "marketing_analysis", "start",
                       message=f"Analyzing distribution across {dimensions}")

        analysis = {}
        for dim in dimensions:
            if dim in self.df.columns:
                dim_analysis = self.df.groupby(dim).agg({
                    "balance": ["sum", "mean", "count"],
                    "dayspastdue": "mean" if "dayspastdue" in self.df.columns else "count"
                }).round(2)
                
                analysis[dim] = {
                    "top_5": dim_analysis.nlargest(5, ("balance", "sum")).to_dict(),
                    "total_records": len(self.df.groupby(dim))
                }

        self.logger.log("INFO", "marketing_analysis", "complete")
        return analysis

    def create_treemap_data(self, primary_dim: str, secondary_dim: str) -> pd.DataFrame:
        if primary_dim not in self.df.columns or secondary_dim not in self.df.columns:
            return pd.DataFrame()

        treemap_data = self.df.groupby([primary_dim, secondary_dim]).agg({
            "balance": "sum",
            "customersegment": "count" if "customersegment" in self.df.columns else "count"
        }).reset_index()
        
        treemap_data.columns = [primary_dim, secondary_dim, "value", "count"]
        return treemap_data

if PLOTLY_AVAILABLE and data_sources:
    for source_name, df in data_sources.items():
        if isinstance(df, pd.DataFrame) and len(df) > 0:
            marketing = MarketingAnalyticsEngine(df, logger)
            
            available_dims = [c for c in df.columns if df[c].nunique() < 50 and df[c].dtype == "object"]
            if len(available_dims) >= 1:
                analysis = marketing.analyze_distribution(available_dims[:3])
                print(f"\nMarketing Analysis for {source_name}:")
                for dim, data in analysis.items():
                    print(f"  {dim}: {data['total_records']} unique values")
            break
else:
    logger.log("WARN", "marketing_analysis", "skip",
              message="Plotly not available for visualization")

## 8. AI-Powered Insights Generation

Generate intelligent business insights using Google Vertex AI (Gemini) when available. Provide rule-based fallback summaries for environments without AI integration.

In [ ]:
class AIInsightsGenerator:
    def __init__(self, logger: StructuredLogger, ai_available: bool = False):
        self.logger = logger
        self.ai_available = ai_available

    def generate_insights(self, kpi_data: Dict[str, Any], audit_report: Dict[str, Any]) -> str:
        self.logger.log("INFO", "ai_insights", "start")

        if self.ai_available:
            return self._generate_ai_insights(kpi_data, audit_report)
        else:
            return self._generate_rule_based_insights(kpi_data, audit_report)

    def _generate_ai_insights(self, kpi_data: Dict[str, Any], audit_report: Dict[str, Any]) -> str:
        try:
            model = GenerativeModel("gemini-1.5-flash")
            prompt = self._build_insights_prompt(kpi_data, audit_report)
            response = model.generate_content(prompt)
            self.logger.log("INFO", "ai_insights", "generated_ai")
            return response.text
        except Exception as e:
            self.logger.log("WARN", "ai_insights", "ai_failed",
                           error=str(e), message="Falling back to rule-based insights")
            return self._generate_rule_based_insights(kpi_data, audit_report)

    def _generate_rule_based_insights(self, kpi_data: Dict[str, Any], audit_report: Dict[str, Any]) -> str:
        insights = []
        insights.append("ABACO Financial Intelligence - Comprehensive Analysis Report")
        insights.append("="*60)

        if kpi_data.get("portfolio_overview"):
            portfolio = kpi_data["portfolio_overview"]
            insights.append(f"\nPortfolio Overview:")
            insights.append(f"Total AUM: ${portfolio.get('total_aum', 0):,.0f}")
            insights.append(f"Active Customers: {portfolio.get('active_customers', 0):,}")
            insights.append(f"Portfolio Utilization: {portfolio.get('portfolio_utilization', 0)*100:.1f}%")

        if kpi_data.get("risk_metrics"):
            risk = kpi_data["risk_metrics"]
            insights.append(f"\nRisk Profile:")
            insights.append(f"Default Rate (90+ DPD): {risk.get('default_rate_90plus', 0)*100:.2f}%")
            insights.append(f"Avg Days Past Due: {risk.get('avg_days_past_due', 0):.1f}")

        if audit_report.get("overall_score"):
            insights.append(f"\nData Quality Score: {audit_report['overall_score']:.1f}/100")

        self.logger.log("INFO", "ai_insights", "generated_rule_based")
        return "\n".join(insights)

    def _build_insights_prompt(self, kpi_data: Dict[str, Any], audit_report: Dict[str, Any]) -> str:
        return f"""Analyze the following financial portfolio data and provide strategic insights:
        
KPI Data: {json.dumps(kpi_data, indent=2)}

Data Quality Report: {json.dumps(audit_report, indent=2)}

Provide:
1. Portfolio health assessment
2. Risk analysis and concerns
3. Growth opportunities
4. Data quality recommendations
5. Strategic recommendations
"""

ai_generator = AIInsightsGenerator(logger, AI_AVAILABLE)

if data_sources:
    for source_name, df in data_sources.items():
        if isinstance(df, pd.DataFrame) and len(df) > 0:
            kpi_engine = ComprehensiveKPIEngine(df, logger)
            result = kpi_engine.calculate_all_kpis()
            
            if result["status"] == "success":
                insights = ai_generator.generate_insights(result["kpis"], audit_report)
                print("\n" + insights)
                break

## 9. Production Export & Figma Integration

Generate flattened fact tables with dimension and metric lists. Export data in formats compatible with Figma and other BI platforms. Create standardized data exports for downstream consumption.

In [ ]:
class ExportEngine:
    def __init__(self, logger: StructuredLogger):
        self.logger = logger
        self.export_dir = Path("/workspaces/nextjs-with-supabase/data/exports")
        self.export_dir.mkdir(parents=True, exist_ok=True)

    def create_flattened_fact_table(self, df: pd.DataFrame, source_name: str) -> pd.DataFrame:
        self.logger.log("INFO", "export", "creating_fact_table",
                       message=f"Creating fact table from {source_name}")

        fact_table = df.copy()
        fact_table["export_date"] = datetime.now(timezone.utc).isoformat()
        fact_table["source"] = source_name

        return fact_table

    def create_figma_export(self, kpi_data: Dict[str, Any], 
                           audit_report: Dict[str, Any]) -> Dict[str, Any]:
        self.logger.log("INFO", "export", "creating_figma_export")

        figma_data = {
            "metadata": {
                "export_date": datetime.now(timezone.utc).isoformat(),
                "platform": "ABACO Financial Intelligence",
                "version": "3.0.0"
            },
            "metrics": self._flatten_metrics(kpi_data),
            "dimensions": self._extract_dimensions(kpi_data),
            "quality_score": audit_report.get("overall_score", 0)
        }

        return figma_data

    def _flatten_metrics(self, kpi_data: Dict[str, Any]) -> List[Dict[str, Any]]:
        metrics = []
        
        for category, values in kpi_data.items():
            if isinstance(values, dict) and category != "segment_analysis":
                for metric_name, metric_value in values.items():
                    if not isinstance(metric_value, (dict, list)):
                        metrics.append({
                            "category": category,
                            "name": metric_name,
                            "value": float(metric_value) if isinstance(metric_value, (int, float, np.number)) else metric_value,
                            "type": "metric"
                        })

        return metrics

    def _extract_dimensions(self, kpi_data: Dict[str, Any]) -> List[Dict[str, Any]]:
        dimensions = []
        
        if "dimensional_analysis" in kpi_data:
            for dim_name, values in kpi_data["dimensional_analysis"].items():
                dimensions.append({
                    "name": dim_name,
                    "values": list(values.keys())[:10],
                    "type": "dimension"
                })

        return dimensions

    def export_to_json(self, data: Any, filename: str) -> str:
        filepath = self.export_dir / filename
        
        def serialize(obj):
            if isinstance(obj, (np.integer, np.floating)):
                return float(obj)
            elif isinstance(obj, pd.DataFrame):
                return obj.to_dict(orient="records")
            elif pd.isna(obj):
                return None
            raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

        with open(filepath, "w") as f:
            json.dump(data, f, indent=2, default=serialize)

        self.logger.log("INFO", "export", "saved",
                       message=f"Exported to {filepath}")

        return str(filepath)

export_engine = ExportEngine(logger)

if data_sources:
    for source_name, df in data_sources.items():
        if isinstance(df, pd.DataFrame) and len(df) > 0:
            kpi_engine = ComprehensiveKPIEngine(df, logger)
            result = kpi_engine.calculate_all_kpis()
            
            if result["status"] == "success":
                figma_export = export_engine.create_figma_export(result["kpis"], audit_report)
                figma_file = export_engine.export_to_json(figma_export, "figma_export.json")
                
                fact_table = export_engine.create_flattened_fact_table(df, source_name)
                csv_file = export_engine.export_to_json(fact_table, "fact_table.json")
                
                print(f"\nExports created:")
                print(f"  Figma: {figma_file}")
                print(f"  Fact Table: {csv_file}")
                break

## 10. Multi-Agent System Integration

Integrate the ABACO multi-agent orchestration system for coordinated analysis. The Supervisor agent delegates to specialized Data Extraction and Quantitative Analysis agents, enabling hierarchical workflow management.

In [ ]:
import sys
sys.path.insert(0, "/workspaces/nextjs-with-supabase/lib/agents")

try:
    from langgraph_agents import (
        AgentOrchestrator, AgentContext, AgentRole, 
        DataExtractionAgent, QuantitativeAnalysisAgent, SupervisorAgent
    )
    AGENTS_AVAILABLE = True
except ImportError as e:
    logger.log("WARN", "multi_agent_setup", "import_failed",
              message=f"Agent module not available: {str(e)}")
    AGENTS_AVAILABLE = False

if AGENTS_AVAILABLE:
    orchestrator = AgentOrchestrator()
    orchestrator.setup_core_agents()
    
    logger.log("INFO", "multi_agent_setup", "complete",
              message="Multi-agent orchestrator initialized",
              metadata={"agents": ["Supervisor", "DataExtraction", "QuantitativeAnalysis"]})
    print("Multi-Agent System Ready:")
    print("  - Supervisor Agent: Query routing and task coordination")
    print("  - Data Extraction Agent: Data loading and normalization")
    print("  - Quantitative Analysis Agent: Metrics and trend calculation")
else:
    print("Multi-Agent System: Not available (async execution requires specific environment)")

## 11. Advanced Hybrid Alert System

Implement sophisticated alert detection combining rule-based statistical analysis with AI-powered severity classification and remediation suggestions. Detects anomalies across multiple variables with contextual risk scoring.

In [ ]:
class HybridAlertSystem:
    def __init__(self, logger: StructuredLogger, ai_available: bool = False):
        self.logger = logger
        self.ai_available = ai_available
        self.alert_history = []
        self.thresholds = self._initialize_thresholds()

    def _initialize_thresholds(self) -> Dict[str, Dict[str, float]]:
        return {
            "default_rate_90plus": {"warning": 0.05, "critical": 0.10},
            "utilization_ratio": {"warning": 0.75, "critical": 0.90},
            "days_past_due_avg": {"warning": 15, "critical": 30},
            "delinquent_aum_pct": {"warning": 0.08, "critical": 0.15},
            "concentration_risk": {"warning": 0.30, "critical": 0.50}
        }

    def detect_alerts(self, df: pd.DataFrame, kpi_data: Dict[str, Any]) -> pd.DataFrame:
        self.logger.log("INFO", "alert_detection", "start",
                       message="Starting hybrid alert detection")

        alerts = []

        if "risk_metrics" in kpi_data:
            risk = kpi_data["risk_metrics"]
            
            default_90 = risk.get("default_rate_90plus", 0)
            alerts.extend(self._evaluate_metric(
                "default_rate_90plus", default_90, risk_context={"total_customers": len(df)}
            ))

            avg_dpd = risk.get("avg_days_past_due", 0)
            alerts.extend(self._evaluate_metric(
                "days_past_due_avg", avg_dpd, risk_context={"median_dpd": df["dayspastdue"].median()}
            ))

            delinquent_aum = risk.get("delinquent_aum", 0)
            total_aum = kpi_data["portfolio_overview"].get("total_aum", 1)
            delinquent_pct = delinquent_aum / max(total_aum, 1)
            alerts.extend(self._evaluate_metric(
                "delinquent_aum_pct", delinquent_pct, risk_context={"delinquent_aum": delinquent_aum}
            ))

        alerts.extend(self._detect_concentration_risk(df))
        alerts.extend(self._detect_volatility_anomalies(df))

        alerts_df = pd.DataFrame(alerts)

        if self.ai_available and len(alerts_df) > 0:
            alerts_df = self._augment_with_ai_classification(alerts_df)

        self.alert_history.extend(alerts)

        self.logger.log("INFO", "alert_detection", "complete",
                       message=f"Detected {len(alerts)} alerts",
                       metadata={
                           "critical_alerts": len([a for a in alerts if a.get("severity") == "critical"]),
                           "warning_alerts": len([a for a in alerts if a.get("severity") == "warning"])
                       })

        return alerts_df

    def _evaluate_metric(self, metric_name: str, value: float, 
                        risk_context: Dict[str, Any]) -> List[Dict[str, Any]]:
        alerts = []
        threshold = self.thresholds.get(metric_name, {})

        if value >= threshold.get("critical", float('inf')):
            severity = "critical"
            probability = min(value / threshold["critical"], 1.0)
        elif value >= threshold.get("warning", float('inf')):
            severity = "warning"
            probability = min((value - threshold["warning"]) / (threshold["critical"] - threshold["warning"]), 1.0)
        else:
            severity = "info"
            probability = 0

        if severity != "info":
            alerts.append({
                "timestamp": datetime.now(timezone.utc).isoformat(),
                "variable": metric_name,
                "value": float(value),
                "threshold_warning": float(threshold.get("warning", 0)),
                "threshold_critical": float(threshold.get("critical", 0)),
                "severity": severity,
                "probability": float(probability),
                "risk_context": risk_context,
                "ai_classification": None,
                "remediation_suggested": self._suggest_remediation(metric_name, severity)
            })

        return alerts

    def _detect_concentration_risk(self, df: pd.DataFrame) -> List[Dict[str, Any]]:
        alerts = []
        
        for col in df.columns:
            if df[col].nunique() < 20 and df[col].dtype == "object":
                concentration = (df[col].value_counts().iloc[0] / len(df))
                if concentration > 0.30:
                    severity = "critical" if concentration > 0.50 else "warning"
                    alerts.append({
                        "timestamp": datetime.now(timezone.utc).isoformat(),
                        "variable": f"concentration_risk_{col}",
                        "value": float(concentration),
                        "threshold_warning": 0.30,
                        "threshold_critical": 0.50,
                        "severity": severity,
                        "probability": min(concentration / 0.50, 1.0),
                        "risk_context": {"top_value": str(df[col].value_counts().index[0])},
                        "ai_classification": None,
                        "remediation_suggested": f"Diversify {col} exposure"
                    })

        return alerts

    def _detect_volatility_anomalies(self, df: pd.DataFrame) -> List[Dict[str, Any]]:
        alerts = []
        numeric_cols = df.select_dtypes(include=[np.number]).columns

        for col in numeric_cols[:3]:
            if len(df[col].dropna()) > 3:
                values = df[col].dropna()
                mean, std = values.mean(), values.std()
                
                outlier_count = ((values > mean + 3*std) | (values < mean - 3*std)).sum()
                outlier_pct = outlier_count / len(values)

                if outlier_pct > 0.05:
                    alerts.append({
                        "timestamp": datetime.now(timezone.utc).isoformat(),
                        "variable": f"volatility_anomaly_{col}",
                        "value": float(outlier_pct),
                        "threshold_warning": 0.05,
                        "threshold_critical": 0.10,
                        "severity": "warning" if outlier_pct < 0.10 else "critical",
                        "probability": min(outlier_pct / 0.10, 1.0),
                        "risk_context": {"outlier_count": int(outlier_count), "mean": float(mean), "std": float(std)},
                        "ai_classification": None,
                        "remediation_suggested": f"Investigate {outlier_count} anomalies in {col}"
                    })

        return alerts

    def _augment_with_ai_classification(self, alerts_df: pd.DataFrame) -> pd.DataFrame:
        if not self.ai_available:
            return alerts_df

        try:
            model = GenerativeModel("gemini-1.5-flash")
            
            for idx, row in alerts_df.iterrows():
                prompt = f"""Classify the severity and provide business context for this financial alert:
                Variable: {row['variable']}
                Value: {row['value']}
                Threshold: {row['threshold_critical']}
                Provide a one-sentence classification of business impact."""
                
                try:
                    response = model.generate_content(prompt)
                    alerts_df.at[idx, "ai_classification"] = response.text[:100]
                except:
                    pass
        except Exception as e:
            self.logger.log("WARN", "alert_ai_augmentation", "failed",
                           error=str(e))

        return alerts_df

    def _suggest_remediation(self, metric_name: str, severity: str) -> str:
        remediation_map = {
            "default_rate_90plus": "Review high-risk customer segments; increase collection efforts",
            "days_past_due_avg": "Accelerate collection process; review KAM engagement",
            "delinquent_aum_pct": "Increase provisions; restructure high-risk exposures",
            "concentration_risk": "Diversify portfolio exposure to reduce concentration",
            "volatility_anomaly": "Investigate outliers; assess data quality"
        }
        base = remediation_map.get(metric_name, "Review metric and assess risk")
        return f"[{severity.upper()}] {base}"

alert_system = HybridAlertSystem(logger, AI_AVAILABLE)

if data_sources:
    for source_name, df in data_sources.items():
        if isinstance(df, pd.DataFrame) and len(df) > 0:
            kpi_engine = ComprehensiveKPIEngine(df, logger)
            result = kpi_engine.calculate_all_kpis()
            
            if result["status"] == "success":
                alerts_df = alert_system.detect_alerts(df, result["kpis"])
                
                print(f"\nAlert Summary for {source_name}:")
                if len(alerts_df) > 0:
                    print(f"Total Alerts: {len(alerts_df)}")
                    print(f"Critical: {len(alerts_df[alerts_df['severity']=='critical'])}")
                    print(f"Warning: {len(alerts_df[alerts_df['severity']=='warning'])}")
                    print(f"\nTop Alerts:")
                    for _, alert in alerts_df.head(5).iterrows():
                        print(f"  [{alert['severity'].upper()}] {alert['variable']}: {alert['value']:.2%}")
                        print(f"    Remediation: {alert['remediation_suggested']}")
                else:
                    print("No alerts detected")
                break

## 12. Advanced Growth Models with ARIMA & Monte Carlo

Implement sophisticated growth forecasting using ARIMA models, Monte Carlo simulations for uncertainty quantification, and scenario analysis (conservative/baseline/aggressive). Generate confidence intervals and statistical probability distributions for strategic planning.

In [ ]:
class AdvancedGrowthModel:
    def __init__(self, df: pd.DataFrame, logger: StructuredLogger):
        self.df = df
        self.logger = logger
        self.try_import_statsmodels()

    def try_import_statsmodels(self):
        try:
            from statsmodels.tsa.arima.model import ARIMA
            self.arima_available = True
        except ImportError:
            self.arima_available = False
            self.logger.log("WARN", "growth_model", "statsmodels_unavailable",
                           message="ARIMA models not available; using fallback")

    def project_with_scenarios(self, target_growth_rate: float, 
                               projection_months: int = 24) -> Dict[str, Any]:
        self.logger.log("INFO", "growth_projection", "start",
                       message=f"Starting advanced growth projection",
                       metadata={"target_growth": target_growth_rate, "months": projection_months})

        current_aum = float(self.df["balance"].sum())
        current_customers = len(self.df)

        scenarios = {
            "conservative": target_growth_rate * 0.5,
            "baseline": target_growth_rate,
            "aggressive": target_growth_rate * 1.5
        }

        results = {
            "current_state": {
                "aum": current_aum,
                "customers": current_customers
            },
            "scenarios": {}
        }

        for scenario_name, growth_rate in scenarios.items():
            scenario_result = self._project_scenario(
                current_aum, current_customers, growth_rate, projection_months, scenario_name
            )
            results["scenarios"][scenario_name] = scenario_result

        if self.arima_available:
            arima_result = self._apply_arima_forecast(current_aum, projection_months)
            results["arima_forecast"] = arima_result

        monte_carlo_result = self._monte_carlo_simulation(current_aum, target_growth_rate, projection_months)
        results["monte_carlo"] = monte_carlo_result

        self.logger.log("INFO", "growth_projection", "complete",
                       message="Advanced growth projection completed")

        return results

    def _project_scenario(self, initial_aum: float, initial_customers: int, 
                          growth_rate: float, months: int, scenario_name: str) -> Dict[str, Any]:
        months_array = np.arange(0, months + 1)
        target_aum = initial_aum * (1 + growth_rate / 100)
        
        monthly_aum = np.linspace(initial_aum, target_aum, len(months_array))
        monthly_customers = np.linspace(initial_customers, initial_customers * (1 + growth_rate / 100), len(months_array))

        return {
            "scenario": scenario_name,
            "growth_rate": growth_rate,
            "months": months_array.tolist(),
            "projected_aum": monthly_aum.tolist(),
            "projected_customers": monthly_customers.astype(int).tolist(),
            "final_aum": float(target_aum),
            "monthly_growth_rate": float(growth_rate / 100 / 12)
        }

    def _apply_arima_forecast(self, initial_aum: float, months: int) -> Dict[str, Any]:
        try:
            from statsmodels.tsa.arima.model import ARIMA
            
            historical_data = np.array([initial_aum * (1 + 0.02 * i) for i in range(12)])
            model = ARIMA(historical_data, order=(1, 1, 1))
            results = model.fit()
            
            forecast = results.get_forecast(steps=months)
            forecast_df = forecast.summary_frame()
            
            return {
                "model_type": "ARIMA(1,1,1)",
                "forecast_values": forecast_df['mean'].tolist(),
                "confidence_upper_95": forecast_df['mean_ci_upper'].tolist(),
                "confidence_lower_95": forecast_df['mean_ci_lower'].tolist(),
                "aic": float(results.aic),
                "bic": float(results.bic)
            }
        except Exception as e:
            self.logger.log("WARN", "arima_forecast", "failed", error=str(e))
            return {"status": "failed", "error": str(e)}

    def _monte_carlo_simulation(self, initial_aum: float, growth_rate: float, 
                                months: int, simulations: int = 1000) -> Dict[str, Any]:
        monthly_growth = growth_rate / 100 / 12
        volatility = 0.05

        paths = np.zeros((simulations, months + 1))
        paths[:, 0] = initial_aum

        for sim in range(simulations):
            for month in range(1, months + 1):
                random_return = np.random.normal(monthly_growth, volatility)
                paths[sim, month] = paths[sim, month - 1] * (1 + random_return)

        percentiles = {
            "p5": np.percentile(paths, 5, axis=0).tolist(),
            "p25": np.percentile(paths, 25, axis=0).tolist(),
            "p50": np.percentile(paths, 50, axis=0).tolist(),
            "p75": np.percentile(paths, 75, axis=0).tolist(),
            "p95": np.percentile(paths, 95, axis=0).tolist()
        }

        return {
            "simulations": simulations,
            "volatility": volatility,
            "percentiles": percentiles,
            "final_aum_distribution": {
                "mean": float(np.mean(paths[:, -1])),
                "std": float(np.std(paths[:, -1])),
                "min": float(np.min(paths[:, -1])),
                "max": float(np.max(paths[:, -1]))
            }
        }

if data_sources:
    for source_name, df in data_sources.items():
        if isinstance(df, pd.DataFrame) and len(df) > 0 and "balance" in df.columns:
            advanced_model = AdvancedGrowthModel(df, logger)
            projections = advanced_model.project_with_scenarios(target_growth_rate, projection_months=24)
            
            print(f"\nAdvanced Growth Projections for {source_name}:")
            print(f"Current AUM: ${projections['current_state']['aum']:,.0f}")
            print(f"\nScenario Analysis:")
            for scenario_name, scenario in projections["scenarios"].items():
                final_aum = scenario["final_aum"]
                gain = final_aum - projections["current_state"]["aum"]
                print(f"  {scenario_name.upper()}: ${final_aum:,.0f} (gain: ${gain:,.0f})")
            
            if "monte_carlo" in projections:
                mc = projections["monte_carlo"]
                print(f"\nMonte Carlo Simulation ({mc.get('simulations', 'N/A')} paths):")
                print(f"  Final AUM - Mean: ${mc['final_aum_distribution']['mean']:,.0f}")
                print(f"  Final AUM - StdDev: ${mc['final_aum_distribution']['std']:,.0f}")
                print(f"  Range: ${mc['final_aum_distribution']['min']:,.0f} to ${mc['final_aum_distribution']['max']:,.0f}")
            break

## 13. Enhanced KPI Analysis with Deeper Segmentation

Extended KPI calculation including cross-dimensional analysis, correlation matrices, stress testing capabilities, and comprehensive risk decomposition by customer segment, region, and product line.

In [ ]:
class EnhancedKPIEngine(ComprehensiveKPIEngine):
    def calculate_extended_kpis(self) -> Dict[str, Any]:
        base_kpis = self.calculate_all_kpis()["kpis"]
        
        self.logger.log("INFO", "enhanced_kpis", "start",
                       message="Calculating extended KPIs")

        extended = {
            **base_kpis,
            "cross_dimensional_analysis": self._cross_dimensional_analysis(),
            "correlation_matrix": self._calculate_correlations(),
            "stress_testing": self._run_stress_tests(),
            "risk_decomposition": self._decompose_risk()
        }

        self.logger.log("INFO", "enhanced_kpis", "complete",
                       message="Extended KPI calculation completed")

        return {"status": "success", "kpis": extended}

    def _cross_dimensional_analysis(self) -> Dict[str, Any]:
        analysis = {}
        
        segment_col = next((c for c in self.data.columns if "segment" in c.lower()), None)
        region_col = next((c for c in self.data.columns if "region" in c.lower()), None)
        
        if segment_col and region_col:
            for segment in self.data[segment_col].unique():
                for region in self.data[region_col].unique():
                    subset = self.data[(self.data[segment_col] == segment) & (self.data[region_col] == region)]
                    if len(subset) > 0:
                        key = f"{segment}_{region}"
                        analysis[key] = {
                            "count": len(subset),
                            "aum": float(subset["balance"].sum()),
                            "default_rate": float((subset["dayspastdue"] >= 90).sum() / len(subset))
                        }

        return analysis

    def _calculate_correlations(self) -> Dict[str, float]:
        correlations = {}
        numeric_cols = self.data.select_dtypes(include=[np.number]).columns
        
        if "balance" in numeric_cols and "dayspastdue" in numeric_cols:
            corr = self.data["balance"].corr(self.data["dayspastdue"])
            correlations["balance_dpd_correlation"] = float(corr)

        if "balance" in numeric_cols and "creditlimit" in numeric_cols:
            corr = self.data["balance"].corr(self.data["creditlimit"])
            correlations["balance_credit_correlation"] = float(corr)

        return correlations

    def _run_stress_tests(self) -> Dict[str, Dict[str, float]]:
        stress_results = {}
        
        current_default_90 = (self.data["dayspastdue"] >= 90).sum() / len(self.data)
        
        stress_scenarios = {
            "dpd_increase_50pct": 1.5,
            "dpd_increase_100pct": 2.0,
            "dpd_increase_150pct": 2.5
        }

        for scenario_name, multiplier in stress_scenarios.items():
            stressed_dpd = self.data["dayspastdue"] * multiplier
            stressed_default_90 = (stressed_dpd >= 90).sum() / len(self.data)
            stress_results[scenario_name] = {
                "base_default_rate": float(current_default_90),
                "stressed_default_rate": float(stressed_default_90),
                "rate_increase": float(stressed_default_90 - current_default_90),
                "multiplier": multiplier
            }

        return stress_results

    def _decompose_risk(self) -> Dict[str, Dict[str, float]]:
        decomposition = {}
        
        segment_col = next((c for c in self.data.columns if "segment" in c.lower()), None)
        if segment_col:
            total_aum = self.data["balance"].sum()
            
            for segment in self.data[segment_col].unique():
                segment_data = self.data[self.data[segment_col] == segment]
                segment_aum = segment_data["balance"].sum()
                segment_default = (segment_data["dayspastdue"] >= 90).sum() / len(segment_data) if len(segment_data) > 0 else 0
                
                risk_contribution = (segment_aum * segment_default) / total_aum
                decomposition[str(segment)] = {
                    "aum_pct": float(segment_aum / total_aum),
                    "default_rate": float(segment_default),
                    "risk_contribution": float(risk_contribution)
                }

        return decomposition

if data_sources:
    for source_name, df in data_sources.items():
        if isinstance(df, pd.DataFrame) and len(df) > 0:
            enhanced_kpi = EnhancedKPIEngine(df, logger)
            result = enhanced_kpi.calculate_extended_kpis()
            
            if result["status"] == "success":
                kpis = result["kpis"]
                print(f"\nEnhanced KPI Analysis for {source_name}:")
                print(f"\nStress Testing Results:")
                if "stress_testing" in kpis:
                    for scenario, metrics in kpis["stress_testing"].items():
                        print(f"  {scenario}:")
                        print(f"    Base Default Rate: {metrics['base_default_rate']:.2%}")
                        print(f"    Stressed Default Rate: {metrics['stressed_default_rate']:.2%}")
                        print(f"    Impact: +{metrics['rate_increase']:.2%}")
                break

## 14. Session Summary & Logging Report

Generate comprehensive session summary with all operations executed, processing metrics, and data quality indicators.

In [ ]:
print("\n" + "="*70)
print("ABACO FINANCIAL INTELLIGENCE PLATFORM - SESSION SUMMARY")
print("="*70)

logger.log("INFO", "session", "summary", message="Generating session summary")

summary = logger.get_summary()

print(f"\nSession ID: {summary['session_id']}")
print(f"Total Operations: {summary['total_logs']}")

print(f"\nOperation Breakdown:")
for operation, count in sorted(summary['by_operation'].items()):
    print(f"  {operation}: {count}")

print(f"\nLog Level Distribution:")
for level, count in sorted(summary['by_level'].items()):
    print(f"  {level}: {count}")

if summary['errors']:
    print(f"\nErrors Encountered: {len(summary['errors'])}")
    for error_log in summary['errors'][:5]:
        print(f"  - {error_log['operation']}: {error_log['error']}")
else:
    print("\nNo errors encountered during execution")

print(f"\nData Sources Processed:")
for source in data_sources.keys():
    print(f"  - {source}")

print(f"\nData Quality Assessment: {audit_report.get('overall_score', 0):.1f}/100")

logger.log("INFO", "session", "complete", message="Session completed successfully")
print(f"\nSession Complete - {datetime.now(timezone.utc).isoformat()}")